In [1]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [2]:
MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [3]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


#### Read in documents using LangChain's loaders

In [4]:
folders = glob.glob("heathprocact_docs/*")
text_loader_kwargs = {'encoding': 'utf-8'}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [6]:
len(chunks)

5

In [7]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: vendors, contact, ESG, about, product


In [8]:
embeddings = OpenAIEmbeddings()

### Create vectorstore

In [11]:
vectorstore = FAISS.from_documents(chunks, embedding=embeddings)

total_vectors = vectorstore.index.ntotal
dimensions = vectorstore.index.d

print(f"There are {total_vectors} vectors with {dimensions:,} dimensions in the vector store")

There are 5 vectors with 1,536 dimensions in the vector store


### Chat Application (RAG pipeline with LangChain)

In [12]:
## create a new Chat with OpenAI (model)
llm = ChatOpenAI(temperature = 0.7, 
                 model_name = MODEL)

## set up the conversation memory for the chat (Memory)
memory = ConversationBufferMemory(
    memory_key = 'chat_history', 
    return_messages = True)

## the retriever is an abstraction over the VectorStore that will be used during RAG (Retreiver)
retriever = vectorstore.as_retriever()
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm = llm, 
    retriever = retriever, 
    memory = memory)

/var/folders/27/gbz_5drj4l13l6x8mnk54j1w0000gn/T/ipykernel_3563/101531323.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [14]:
query = "Can you describe Heathprocact in a few sentences"
result = conversation_chain.invoke({"question":query})
print(result["answer"])

Heathprocact is a cutting-edge company that specializes in geospatial data, AI, and machine learning models. It focuses on developing AI agents and ESG indicators related to geospatial data, enabling businesses and organizations to make smarter and more sustainable decisions. The company's mission is to bridge technology and sustainability through geospatial intelligence, empowering governments, NGOs, and businesses to address challenges in climate change, natural disasters, and sustainable development.


In [15]:
def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [16]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
